In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx
import networkx as nx
import matplotlib.pyplot as plt
import time
import random
import copy

/Users/atharvamendhulkar/Desktop/GNN/gnn-env/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/atharvamendhulkar/Desktop/GNN/gnn-env/lib/python3.12/site-packages/libpyg.so, 0x0006): Library not loaded: /Library/Frameworks/Python.framework/Versions/3.12/Python
  Referenced from: <441E30E4-F1D4-325A-924A-8C4E5BD0FA29> /Users/atharvamendhulkar/Desktop/GNN/gnn-env/lib/python3.12/site-packages/libpyg.so
  Reason: tried: '/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/Users/atharvamendhulkar/Desktop/GNN/gnn-env/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred whi

In [ ]:
# Run this in a Jupyter notebook cell
%pip install torch torchvision torchaudio torch-geometric networkx matplotlib numpy



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Ensure `pyg_graphs` is defined (pick first available batch if needed)
if 'pyg_graphs' not in globals():
    if 'pyg_graphs_all' in globals() and len(pyg_graphs_all) > 0:
        first_key = list(pyg_graphs_all.keys())[0]
        pyg_graphs = pyg_graphs_all[first_key][0]  # first type, first size batch
        print(f"Defined pyg_graphs from pyg_graphs_all['{first_key}'][0] with {len(pyg_graphs)} snapshots")
    else:
        raise RuntimeError('pyg_graphs not found. Run the graph generation cells first.')


In [ ]:
prev_colors = None
final_colors = None
feature_dim = 1 + num_colors  # degree + previous colors
model = TemporalGCN(in_channels=feature_dim, hidden_channels=32, num_colors=num_colors)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
markdown
markdown
### Demo: View generated dynamic coloring animation
After running `src/animate_dynamic_coloring.py` the demo GIF is saved under `results/dynamic_coloring_demo.gif`.
You can re-run the script from the command line or the notebook to regenerate the animation with different parameters.
code
python
# Display the generated GIF (path relative to this notebook)
from IPython.display import Image, display, HTML
display(HTML("<h4>Dynamic Coloring Demo</h4>"))
display(Image(filename='../results/dynamic_coloring_demo.gif'))
    
    edge_index = g.edge_index
    num_nodes = g.num_nodes
    
    device = next(model.parameters()).device

    # Node features
    degrees = torch.zeros(num_nodes, dtype=torch.float, device=device)
    degrees.scatter_add_(0, edge_index[0], torch.ones(edge_index.size(1), device=device))
    degrees = degrees.unsqueeze(1)
    
    prev_colors_feat = (
        F.one_hot(prev_colors, num_classes=num_colors).float()
        if prev_colors is not None
        else torch.zeros(num_nodes, num_colors, device=device)
    )
    
    x = torch.cat([degrees, prev_colors_feat], dim=1)

    # GNN forward + training
    optimizer.zero_grad()
    logits = model(x, edge_index)
    loss, pred_colors = potts_loss(logits, edge_index, prev_colors)
    loss.backward()
    optimizer.step()

    # Hybrid repair
    final_colors = hybrid_repair(pred_colors, edge_index, num_colors)

    # Metrics
    conflicts = count_conflicts(final_colors, edge_index)
    recolors = count_recolors(prev_colors, final_colors)
    used_colors = colors_used(final_colors)
    latency = time.time() - start_time

    # Record
    history["conflicts"].append(conflicts)
    history["recolors"].append(recolors)
    history["colors_used"].append(used_colors)
    history["latency"].append(latency)

    # Update prev_colors & detach hidden state
    prev_colors = final_colors
    if model.hidden_state is not None:
        model.hidden_state = model.hidden_state.detach()

    print(f"Step {t}: loss={loss.item():.4f}, conflicts={conflicts}, recolors={recolors}, ",
          f"colors_used={used_colors}, latency={latency:.4f}s")

